# We will be removing the keys from the json objects, as they might impact the score.

In [ ]:
import pandas as pd

In [ ]:
human_responses = []
gpt_responses = []

In [ ]:
import json
with open("GPT-4o-results/GPT4o-results-19.jsonl", "r") as fp:
	for line in fp.readlines():
		human_responses.append({"testcases": json.loads(line)["testcases"]})
		gpt_responses.append({"testcases": json.loads(line)["GPT4o_testcases"]})

In [ ]:
print(human_responses[0])
print(gpt_responses[0])

In [ ]:
with open("GPT-4o-results/GPT4o-results-20.jsonl", "r") as fp:
	for line in fp.readlines():
		human_responses.append({"testcases": json.loads(line)["testcases"]})
		gpt_responses.append({"testcases": json.loads(line)["GPT4o_testcases"]})

In [ ]:
print(len(human_responses))
print(len(gpt_responses))

# Remove fixed keys

In [ ]:
def remove_keys(d):
    """
    remove all first level keys from d, convert it to a string
    """
    return str(list(d.values()))[1:-1]

In [ ]:
for idx, tcs in enumerate(human_responses):
	new_tcs = []
	if not isinstance(tcs["testcases"], list):
		tcs["testcases"] = [tcs["testcases"]]
	for tc in tcs["testcases"]:
		new_tcs.append(remove_keys(tc))
	# break
	human_responses[idx] = "; ".join(new_tcs)

In [ ]:
for idx, tcs in enumerate(gpt_responses):
	new_tcs = []
	if not isinstance(tcs["testcases"], list):
		tcs["testcases"] = [tcs["testcases"]]
	for tc in tcs["testcases"]:
		new_tcs.append(remove_keys(tc))
	# break
	gpt_responses[idx] = "; ".join(new_tcs)

In [ ]:
print(human_responses[0])
print(gpt_responses[0])

In [ ]:
scores = {}

In [ ]:
import evaluate

rouge = evaluate.load("rouge")
	
scores["rouge"] = rouge.compute(predictions=gpt_responses, references=human_responses)

print(f"Finetuned scores: {scores["rouge"]}")

In [ ]:
import evaluate

bleu = evaluate.load("bleu")
	
scores["blue"] = bleu.compute(predictions=gpt_responses, references=human_responses)

print(f"Finetuned scores: {scores["blue"]}")

In [ ]:
import evaluate

bert = evaluate.load("bertscore")
	
scores["bert"] = bert.compute(predictions=gpt_responses, references=human_responses, lang="en")

print(f"Finetuned scores: {scores["bert"]}")

In [ ]:
scores["bert"].keys()

In [ ]:
import numpy as np

print(np.mean(scores["bert"]["precision"]))
print(np.mean(scores["bert"]["recall"]))
print(np.mean(scores["bert"]["f1"]))

In [ ]:
scores["bert"]["precision"] = np.mean(scores["bert"]["precision"])
scores["bert"]["recall"] = np.mean(scores["bert"]["recall"])
scores["bert"]["f1"] = np.mean(scores["bert"]["f1"])

In [ ]:
import json
with open(f"gpt4o_scores_final_fixed.json", "w") as fp:
	json.dump(scores, fp)